In [88]:
import hashlib
import random

In [90]:
def exp_mod(a: int , b: int , n: int) -> int:
    # Argumentos:
    #    a: int - a >= 0
    #    b: int - b >= 0
    #    n: int - n > 0
    # Retorna:
    #    int - a**b en  modulo n
    if b == 1:
        return a % n
    if b % 2 == 0: 
        val = exp_mod(a, b//2, n)
        return val*val % n
    val = exp_mod(a, (b-1)//2, n)
    return val*val*a % n

In [91]:
def  alg_ext_euclides(a: int , b: int) -> (int , int , int):
    # Argumentos :
    #    a: int
    #    b: int - a >= b >= 0 y a > 0
    # Retorna :
    #    (int , int , int) - maximo  comun  divisor  MCD(a,b) entre a y b,
    #    y numeros  enteros s y t tales  que MCD(a,b) = s*a + t*b
    values = {'s_i-1': 1, 's_i': 0, 's_i+1': 0, 't_i-1': 0, 't_i': 1, 
              't_i+1': 0, 'r_i-1': a, 'r_i': b, 'r_i+1': 0}
    while values['r_i'] != 0:
        values['q_i'] = values['r_i-1'] // values['r_i']
        values['r_i+1'] = values['r_i-1'] % values['r_i']
        values['s_i+1'] = values['s_i-1'] - values['q_i']*values['s_i']
        values['t_i+1'] = values['t_i-1'] - values['q_i']*values['t_i']
        values['r_i-1'] = values['r_i']
        values['s_i-1'] = values['s_i']
        values['t_i-1'] = values['t_i']
        values['r_i'] = values['r_i+1']
        values['s_i'] = values['s_i+1']
        values['t_i'] = values['t_i+1']
    return (values['r_i-1'],values['s_i-1'],values['t_i-1'])

In [92]:
def inverso(a: int , n: int) -> int:
    # Argumentos:
    #    a: int - a >= 1
    #    n: int - n >= 2, a y n son  primos  relativos
    # Retorna:
    #    int - inverso  de a en  modulo n
    inverso = alg_ext_euclides(n, a)[2]
    if inverso < 0:
        return inverso + n
    return inverso

In [89]:
def get_parameters():
    with open('grupo.txt') as file:
        lines = file.readlines()
    values = {}
    for line in lines:
        sep = line.split('=')
        values[sep[0]] = sep[1]
    return int(values['p'], 16), int(values['g'], 16), int(values['q'], 16)

In [303]:
def  generar_clave_ElGamal():
    # Retorna :
    #    Genera  una  clave  privada y una  clave  publica  segun el  protocolo
    #    critografico  ElGamal , para el  grupo  almacenado  en  grupo.txt.
    #    Almacena  la  clave  privada  en  private_key.txt , y la  clave  publica
    #    en  public_key.txt.
    p, g, q = get_parameters()
    x = random.randint(1, q - 1)
    y = exp_mod(g, x, p)
    with open('private_key.txt', 'w') as file:
        file.write(str(x))
    with open('public_key.txt', 'w') as file:
        file.write(str(y))

In [304]:
def  firmar_Schnorr(m: str) -> (int ,int):
    # Argumentos :
    #    m: str - mensaje
    # Retorna :
    #    (int ,int) - firma de  Schnorr (e,s) del  mensaje m segun la  clave
    #    privada  almacenada  en  private_key.txt , para el  grupo  almacenado
    #    en  grupo.txt
    p, g, q = get_parameters()
    with open('private_key.txt', 'r') as file:
        private_key = int(file.readline())
    with open('public_key.txt', 'r') as file:
        public_key = int(file.readline())
    k = random.randint(1, q - 1)
    r = exp_mod(g, k, p)
    e = md5(str(r) + m)
    s = k - private_key * e
    return (e, s)

In [411]:
def  verificar_firma_Schnorr(m: str , firma: (int ,int)) -> bool:
    # Argumentos :
    #    m: str - mensaje
    #    firma: (int ,int) - firma de  Schnorr (e,s) para m
    #Retorna :
    #    bool - retorna  True si para el  usario  con  clave  publica
    #    almacenada  en  public_key.txt , el par (e,s) es una  firma de
    #    Schnorr  correcta  para el  mensaje m, en el  grupo  almacenado  en
    #    grupo.txt. En caso  contrario  retorna  False
    p, g, q = get_parameters()
    e, s = firma
    with open('public_key.txt', 'r') as file:
        public_key = int(file.readline())
    if s < 0:
        s2 = inverso(exp_mod(g, -s, p), p)
        r_prime = (s2 * exp_mod(public_key,e,p)) % p
    else:
        r_prime = (exp_mod(g,s,p) * exp_mod(public_key,e,p)) % p
    return md5(str(r_prime) + m) == e

In [412]:
def  md5(m: str) -> int:
    # Argumentos :
    #    m: str - mensaje
    #Retorna :
    #    int - valor de la  funcion  de hash  MD5  aplicada  sobre m
    hashed_m = hashlib.md5(bytes(m, encoding='utf-8')).hexdigest()
    return int(hashed_m, 16)

In [426]:
generar_clave_ElGamal()

In [427]:
firma = firmar_Schnorr('firma dïgîtál!')

In [428]:
verificar_firma_Schnorr('firma dïgîtál!', firma)

True

In [420]:
firma = firmar_Schnorr('firma dïgîtál!~')

In [421]:
verificar_firma_Schnorr('firma dïgîtál!^', firma)

False